# INM430 Week 10 Practical - Part 1
Scrape London Datastore  
Here we build an inventory of what is available and where  

Part 1 of 3 
1. Get LDS column names
2. Get Health data column names  
3. Cross reference and determine scope  

In [173]:
# helper functions

def getLDSDownloadLinksPageCount():
    # get the number of London Datastore pages we can scrape
    # using the same general format from week 09
    from bs4 import BeautifulSoup
    import urllib.request as ur
    urlToScrape = "https://data.london.gov.uk/dataset"
    r = ur.urlopen(urlToScrape).read()
    soup = BeautifulSoup(r, "lxml")
    # looking for the paging links found near footer
    linkList = soup.find_all('li', attrs={'class': 'dp-search__pagelink'})
    # all being well, the list will look like this (where each line is a list element)
    
    #<li class="dp-search__pagelink dp-search__pagelink--disabled"><span>«</span></li>
    #<li class="dp-search__pagelink dp-search__pagelink--active"><span>1</span></li>
    #<li class="dp-search__pagelink"><a href="/dataset?page=2">2</a></li>
    #<li class="dp-search__pagelink"><a href="/dataset?page=3">3</a></li>
    #<li class="dp-search__pagelink"><a href="/dataset?page=4">4</a></li>
    #<li class="dp-search__pagelink"><span>...</span></li>
    #<li class="dp-search__pagelink"><a href="/dataset?page=78">78</a></li>
    #<li class="dp-search__pagelink"><a href="/dataset?page=2">»</a></li>
    
    # The line we are interested in is the next to last (page 78), number 78 being
    # the text property of the link (a href attribute), which is the 6th element of
    # the linkList list starting from index 0
    try:
        iPagenums = linkList[6].text
    except:
        # string data type for consistency
        iPagenums = "0"

    return int(iPagenums)

def getLDSDownloadLinks(iPagenum):
    from bs4 import BeautifulSoup
    import urllib.request as ur
    urlToScrape = "https://data.london.gov.uk/dataset?page=" + str(iPagenum)
    r = ur.urlopen(urlToScrape).read()
    soup = BeautifulSoup(r, "lxml")
    # look for h3 headers
    linkList = soup.find_all('h3', attrs={'class': 'dp-searchresult__heading'})
    # our return list
    results = []
    for linkListItem in linkList:
        try:
            linkHeader = linkListItem.find('a', attrs={'class': "dp-searchresult__heading-link"})
            name = linkHeader.text
            href = linkHeader['href']
            ldslinks = {
                "name" : name,
                "href" : href,
            }
        except Exception as e:
            print("Error - no links found")
        results.append(ldslinks)
    return results

def getLDSFileDownloadLinkOld(href):
    # get the file download link - note the link might expire
    from bs4 import BeautifulSoup
    import urllib.request as ur
    urlToScrape = "https://data.london.gov.uk" + href
    r = ur.urlopen(urlToScrape).read()
    soup = BeautifulSoup(r, "lxml")
    try:
        downloadlink = soup.find('a', attrs={'class': 'dp-resource__link'})
        filelink = downloadlink['href']
    except:
        filelink = ""
    return filelink

def getLDSFileDownloadLinks(href):
    # get the file download links - pdf, xls, etc, decide later what to do
    from bs4 import BeautifulSoup
    import urllib.request as ur
    href = '/dataset/london-food-strategy-consultation-2018'
    urlToScrape = "https://data.london.gov.uk" + href
    r = ur.urlopen(urlToScrape).read()
    soup = BeautifulSoup(r, "lxml")
    downloads = soup.find_all('div', attrs={'class': 'dp-resource__indented'})
    fileurls = []
    for download in downloads:
        try:
            links = download.find('a', attrs={'class': 'dp-resource__format'})
            fileurl = links['href']
            dictlinks = {
                "fileurl" : fileurl,
            }
            fileurls.append(dictlinks)
        except:
            print("Error occured parsing file download links")
    return fileurls


In [45]:
iPagenums = getLDSDownloadLinksPageCount()
# initialise our links dictionary
links = []
for i in range(1, iPagenums + 1):
    links.extend(getLDSDownloadLinks(i))

In [81]:
for link in links:
    print(link['name'],link['href'], link['url'])
    getLDSFileDownloadLink(href)
    # update links[0]['fileurl'] = "stuff"
# can't use for loop, will have to use an index    

KeyError: 'url'

In [165]:
links = []
for i in range(1, 2):
    links.extend(getLDSDownloadLinks(i))

In [129]:
getLDSFileDownloadLink('/dataset/london-food-strategy-consultation-2018')

In [159]:
# get the file download links - pdf, xls, etc, decide later what to do
from bs4 import BeautifulSoup
import urllib.request as ur
href = '/dataset/london-food-strategy-consultation-2018'
urlToScrape = "https://data.london.gov.uk" + href
r = ur.urlopen(urlToScrape).read()
soup = BeautifulSoup(r, "lxml")
downloads = soup.find_all('div', attrs={'class': 'dp-resource__indented'})
fileurls = []
for download in downloads:
    try:
        links = download.find('a', attrs={'class': 'dp-resource__format'})
        fileurl = links['href']
        dictlinks = {
            "fileurl" : fileurl,
        }
        fileurls.append(dictlinks)
    except:
        print("Error occured parsing file download links")
        
return fileurls
        
for i in range(0, len(fileurls)):
    print(str(i))
    print(fileurls[i]['fileurl'])

0
/download/london-food-strategy-consultation-2018/23e29736-c686-4c8a-8743-0aa23b3f8763/GLA%20consultation%20results%20-%20Unhealthy%20food%20advert%20ban.xls
1
/download/london-food-strategy-consultation-2018/723e7842-6ae5-4b2a-bf82-e146f4668838/GLA%20YouGov%20survey%20May%202018%20advert%20ban.pdf


In [167]:
links = getLDSDownloadLinks(1)

In [168]:
links

[{'name': 'Income Inequality', 'href': '/dataset/income-inequality'},
 {'name': 'HBAI Poverty in London', 'href': '/dataset/hbai-poverty'},
 {'name': 'Gender Pay Gaps in London', 'href': '/dataset/gender-pay-gaps'},
 {'name': 'Equality, Diversity and Inclusion Evidence Base for London',
  'href': '/dataset/equality--diversity-and-inclusion-evidence-base-2018'},
 {'name': 'London Underground Performance Reports',
  'href': '/dataset/london-underground-performance-reports'},
 {'name': 'GLA Poll Results 2018', 'href': '/dataset/gla-poll-results-2018'},
 {'name': 'Employees earning below the London Living Wage (LLW)',
  'href': '/dataset/earning-below-llw'},
 {'name': 'Young Londoners Fund Projects',
  'href': '/dataset/young-londoners-fund-projects'},
 {'name': 'Public Transport Journeys by Type of Transport',
  'href': '/dataset/public-transport-journeys-type-transport'},
 {'name': 'London Food Strategy Consultation 2018',
  'href': '/dataset/london-food-strategy-consultation-2018'}]

In [171]:
for i in range (0, len(links)):
    print(links[i]['name'], links[i]['href'])

Income Inequality /dataset/income-inequality
HBAI Poverty in London /dataset/hbai-poverty
Gender Pay Gaps in London /dataset/gender-pay-gaps
Equality, Diversity and Inclusion Evidence Base for London /dataset/equality--diversity-and-inclusion-evidence-base-2018
London Underground Performance Reports /dataset/london-underground-performance-reports
GLA Poll Results 2018 /dataset/gla-poll-results-2018
Employees earning below the London Living Wage (LLW) /dataset/earning-below-llw
Young Londoners Fund Projects /dataset/young-londoners-fund-projects
Public Transport Journeys by Type of Transport /dataset/public-transport-journeys-type-transport
London Food Strategy Consultation 2018 /dataset/london-food-strategy-consultation-2018


In [175]:
for i in range (0, len(links)):
    links[i]['fileurls'] = getLDSFileDownloadLinks(links[i]['href'])

In [174]:
getLDSFileDownloadLinks('/dataset/london-food-strategy-consultation-2018')

[{'fileurl': '/download/london-food-strategy-consultation-2018/23e29736-c686-4c8a-8743-0aa23b3f8763/GLA%20consultation%20results%20-%20Unhealthy%20food%20advert%20ban.xls'},
 {'fileurl': '/download/london-food-strategy-consultation-2018/723e7842-6ae5-4b2a-bf82-e146f4668838/GLA%20YouGov%20survey%20May%202018%20advert%20ban.pdf'}]

In [177]:
for i in range (0, len(links)):
    print(links[i]['fileurls'])

[{'fileurl': '/download/london-food-strategy-consultation-2018/23e29736-c686-4c8a-8743-0aa23b3f8763/GLA%20consultation%20results%20-%20Unhealthy%20food%20advert%20ban.xls'}, {'fileurl': '/download/london-food-strategy-consultation-2018/723e7842-6ae5-4b2a-bf82-e146f4668838/GLA%20YouGov%20survey%20May%202018%20advert%20ban.pdf'}]
[{'fileurl': '/download/london-food-strategy-consultation-2018/23e29736-c686-4c8a-8743-0aa23b3f8763/GLA%20consultation%20results%20-%20Unhealthy%20food%20advert%20ban.xls'}, {'fileurl': '/download/london-food-strategy-consultation-2018/723e7842-6ae5-4b2a-bf82-e146f4668838/GLA%20YouGov%20survey%20May%202018%20advert%20ban.pdf'}]
[{'fileurl': '/download/london-food-strategy-consultation-2018/23e29736-c686-4c8a-8743-0aa23b3f8763/GLA%20consultation%20results%20-%20Unhealthy%20food%20advert%20ban.xls'}, {'fileurl': '/download/london-food-strategy-consultation-2018/723e7842-6ae5-4b2a-bf82-e146f4668838/GLA%20YouGov%20survey%20May%202018%20advert%20ban.pdf'}]
[{'fileurl